<a href="https://colab.research.google.com/github/Harshitjain18/Smart_chef/blob/main/Smart_chef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!pip install mealpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 55.8 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 817, in _parseNoCache
    loc, tokens = self.parseImpl(instring, pre_loc, doActions)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 2337, in parseImpl
    if instring[loc] == self.firstMatchChar and instring.startswith(
IndexError: string index 

In [ ]:

import numpy as np
from mealpy import FloatVar,HS,BES
from mealpy.optimizer import Optimizer
from tqdm import tqdm
import os

global globepoch=0
class OriginalWaOA(Optimizer):
    """
    The original version of: Walrus Optimization Algorithm (WaOA)

    Links:
        1. https://www.researchgate.net/publication/364684780_Walrus_Optimization_Algorithm_A_New_Bio-Inspired_Metaheuristic_Algorithm

    Notes:
        1. This is somewhat concerning, as there appears to be a high degree of similarity between the source code for this algorithm and the Northern Goshawk Optimization (NGO)
        2. Algorithm design is similar to Zebra Optimization Algorithm (ZOA), Osprey Optimization Algorithm (OOA), Coati Optimization Algorithm (CoatiOA), Siberian Tiger Optimization (STO), Language Education Optimization (LEO), Serval Optimization Algorithm (SOA), Northern Goshawk Optimization (NGO), Fennec Fox Optimization (FFO), Three-periods optimization algorithm (TPOA), Teamwork optimization algorithm (TOA), Pelican Optimization Algorithm (POA), Tasmanian devil optimization (TDO), Archery algorithm (AA), Cat and mouse based optimizer (CMBO)
        3. It may be useful to compare the Matlab code of this algorithm with those of the similar algorithms to ensure its accuracy and completeness.
        4. The article may share some similarities with previous work by the same authors, further investigation may be warranted to verify the benchmark results reported in the papers and ensure their reliability and accuracy.

    Examples
    ~~~~~~~~
    >>> import numpy as np
    >>> from mealpy import FloatVar, WaOA
    >>>
    >>> def objective_function(solution):
    >>>     return np.sum(solution**2)
    >>>
    >>> problem_dict = {
    >>>     "bounds": FloatVar(n_vars=30, lb=(-10.,) * 30, ub=(10.,) * 30, name="delta"),
    >>>     "minmax": "min",
    >>>     "obj_func": objective_function
    >>> }
    >>>
    >>> model = WaOA.OriginalWaOA(epoch=1000, pop_size=50)
    >>> g_best = model.solve(problem_dict)
    >>> print(f"Solution: {g_best.solution}, Fitness: {g_best.target.fitness}")
    >>> print(f"Solution: {model.g_best.solution}, Fitness: {model.g_best.target.fitness}")

    References
    ~~~~~~~~~~
    [1] Trojovský, P., & Dehghani, M. (2022). Walrus Optimization Algorithm: A New Bio-Inspired Metaheuristic Algorithm.
    """
    def __init__(self, epoch: int = 10000, pop_size: int = 100,ppc=0.2,mf=5,flag=0,flag1=0,flag2=0, **kwargs: object) -> None:
        """
        Args:
            epoch (int): maximum number of iterations, default = 10000
            pop_size (int): number of population size, default = 100
        """
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [5, 10000])
        self.set_parameters(["epoch", "pop_size"])
        self.is_parallelizable = False
        self.sort_flag = False
        self.population_propogation_constant=ppc
        self.multi_factor=mf
        self.flag=flag
        self.flag1=flag1
        self.flag2=flag2

    def evolve(self, epoch):
        """
        The main operations (equations) of algorithm. Inherit from Optimizer class

        Args:
            epoch (int): The current iteration
        """
        global globepoch
        globepoch=epoch
        if self.flag1 ==0 or self.flag1==1:
            for idx in range(self.pop_size):
                random=self.generator.random()
                if random <self.population_propogation_constant:
                    sum_variable=0
                    sum_adjusted=0
                    zero_fitness=[]
                    for i in range(self.multi_factor):
                        new_agent=self.pop[idx].solution*(1+(-1+2*self.generator.random())*(self.problem.ub-self.problem.lb)/self.pop_size)
                        f=self.get_target(new_agent).fitness
                        if f == 0:
                            zero_fitness.append(new_agent)
                            continue
                        sum_variable+=new_agent/f
                        sum_adjusted+=f**-1
                    if sum_adjusted==0:
                        continue
                    com=(sum_variable/sum_adjusted)
                    pos_new2 = self.correct_solution(com)
                    com_agent = self.generate_agent(pos_new2)
                    if self.flag2!=0:
                        if self.compare_target(com_agent.target, self.pop[idx].target, self.problem.minmax):
                            self.pop[idx]=com_agent
                    else:
                        self.pop[idx]=com_agent


        if self.flag1==1 or self.flag1==2:
            for idx in range(0, self.pop_size):
                # Phase 1: Feeding strategy (exploration)
                if self.flag ==0:
                    pos_new2=self.pop[idx].solution + self.generator.random()*(self.g_best.solution-(1+self.generator.random())*self.pop[idx].solution)
                    pos_new2 = self.correct_solution(pos_new2)
                    agent2 = self.generate_agent(pos_new2)
                    if self.compare_target(agent2.target, self.pop[idx].target, self.problem.minmax):
                        self.pop[idx]=agent2

                kk = self.generator.permutation(self.pop_size)[0]
                if self.compare_target(self.pop[kk].target, self.pop[idx].target, self.problem.minmax):     # Eq. 4
                    pos_new = self.pop[idx].solution + self.generator.random() * (self.pop[kk].solution -(1+ self.generator.random() )* self.pop[idx].solution)
                else:
                    pos_new = self.pop[idx].solution + self.generator.random() * (self.pop[idx].solution - self.pop[kk].solution)
                pos_new = self.correct_solution(pos_new)
                agent = self.generate_agent(pos_new)
                if self.compare_target(agent.target, self.pop[idx].target, self.problem.minmax):
                    self.pop[idx] = agent

                if self.flag != 0:
                    # PHASE 2 Exploitation
                    LB, UB = self.problem.lb / epoch, self.problem.ub / epoch
                    pos_new = self.pop[idx].solution + LB + (UB - self.generator.random() * LB)     # Eq. 7
                    pos_new = self.correct_solution(pos_new)
                    agent = self.generate_agent(pos_new)
                    if self.compare_target(agent.target, self.pop[idx].target, self.problem.minmax):
                        self.pop[idx] = agent

In [ ]:

import numpy as np
from mealpy import PermutationVar, WOA, Problem

Time_of_order = [[2, 1, 3], [4, 2, 1], [3, 3, 2]]
Time_of_making = [[3, 2, 1], [1, 4, 2], [2, 3, 2]]

n_jobs = len(job_times)
n_machines = len(machine_times)

data = {
    "job_times": Time_of_order,
    "machine_times": Time_of_making,
    "n_jobs": n_jobs,
    "n_machines": n_machines
}

class JobShopProblem(Problem):
    def __init__(self, bounds=None, minmax="min", data=None, **kwargs):
        self.data = data
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, x):
        x_decoded = self.decode_solution(x)
        x = x_decoded["per_var"]
        makespan = np.zeros((self.data["n_jobs"], self.data["n_machines"]))
        for gene in x:
            job_idx = gene // self.data["n_machines"]
            machine_idx = gene % self.data["n_machines"]
            if job_idx == 0 and machine_idx == 0:
                makespan[job_idx][machine_idx] = job_times[job_idx][machine_idx]
            elif job_idx == 0:
                makespan[job_idx][machine_idx] = makespan[job_idx][machine_idx - 1] + job_times[job_idx][machine_idx]
            elif machine_idx == 0:
                makespan[job_idx][machine_idx] = makespan[job_idx - 1][machine_idx] + job_times[job_idx][machine_idx]
            else:
                makespan[job_idx][machine_idx] = max(makespan[job_idx][machine_idx - 1], makespan[job_idx - 1][machine_idx]) + job_times[job_idx][machine_idx]
        return np.max(makespan)


bounds = PermutationVar(valid_set=list(range(0, n_jobs*n_machines)), name="per_var")
problem = JobShopProblem(bounds=bounds, minmax="min", data=data)

model = OriginalWaOA(epoch=100, pop_size=50,flag=1,flag1=1,flag2=1,ppc=1)
model.solve(problem)

print(f"Best agent: {model.g_best}")                    # Encoded solution
print(f"Best solution: {model.g_best.solution}")        # Encoded solution
print(f"Best fitness: {model.g_best.target.fitness}")
print(f"Best real scheduling: {model.problem.decode_solution(model.g_best.solution)}")      # Decoded (Real) solution